In [0]:
import sys

sys.path.append("../include_utils/")

import ipyparallel as ipp
import os, time
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
import dill
from scipy import stats
from IPython.display import display
import geopy

samtools = "/home/cfriedline/gpfs/src/samtools-1.3/samtools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
picard = "/home/cfriedline/gpfs/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/g/src/jdk1.8.0_60/bin/java"
perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

vcfutils = "perl /home/cfriedline/g/src/bcftools-1.3/vcfutils.pl"
vcftools = "/home/cfriedline/bin/vcftools"
bcftools = "/home/cfriedline/gpfs/src/bcftools-1.3/bcftools"
tabix = "/home/cfriedline/gpfs/src/htslib-1.3/tabix"
bgzip = "/home/cfriedline/gpfs/src/htslib-1.3/bgzip"


def setup_r():
    os.environ['R_HOME'] = '/home/cfriedline/g/R3/lib64/R'
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'],
                                                     "/home/cfriedline/lib64")

setup_r()
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
r = robjects.r

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext rpy2.ipython

In [0]:
%%R
varcomp_pop = readRDS("/home/cfriedline/eckertlab/Mitra/dDocent/isect_hierfstat_varcomp.rds")
varcomp_spp = readRDS("/home/cfriedline/eckertlab/Mitra/dDocent/isect_hierfstat_varcomp_nested_in_spp.rds")

In [0]:
def get_r_series(key):
    s = pd.Series(get_r(key))
    s.index = get_r("names(%s)" % key)
    return s

def get_r_df(key):
    df = pd.DataFrame(get_r(key))
    try:
        rname = get_r("rownames(%s)" % key)
        df.index = rname
    except:
        pass
    
    try:
        cname = get_r("colnames(%s)" % key)
        df.columns = cname
    except:
        pass
    
    return df

def get_r(key):
    return r(key)

In [0]:
loc_df_spp = get_r_df('varcomp_spp$loc')
loc_df_pop = get_r_df('varcomp_pop$loc')


In [0]:
def compute_fst(series):
    Hs = series[0]
    Ht = sum(series)
    return Hs/Ht

In [0]:
loci_fst_pop = loc_df_pop.apply(compute_fst, axis=1)
loci_fst_spp = loc_df_spp.apply(compute_fst, axis=1)


In [0]:
loci_fst_spp.describe()

In [0]:
loci_fst_pop.describe()

In [0]:
fst_df = pd.DataFrame(loci_fst_pop).join(pd.DataFrame(loci_fst_spp), rsuffix="_spp", lsuffix="_pop")
fst_df.columns = ["(pop, total)", "(spp, total)"]

In [0]:
sns.set_context("talk")
sns.set_style("white")
sns.regplot("(pop, total)", "(spp, total)", data=fst_df)

In [0]:
spp = fst_df["(spp, total)"].values
pop = fst_df["(pop, total)"].values

In [0]:
slope, intercept, r_value, p_value, std_err = stats.linregress(spp, pop)

In [0]:
print(p_value)